In [1]:
!pip install transformers

In [2]:
!git clone https://github.com/cjGO/yaamformer.git

fatal: destination path 'yaamformer' already exists and is not an empty directory.


In [3]:
from ast import literal_eval
from yaamformer.utils import get_dataset


In [4]:
def get_dataset(file_location):# Using readline()
    file1 = open(file_location, 'r')
    count = 0

    Seqs = []
    Labels = []

    while True:
        count += 1
        #print(count)
        # Get next line from file
        line = file1.readline()

        if not line:
            break

        labels = literal_eval(line.split('\t')[2])
        seqs = line.split('\t')[5]
        seqs = seqs.replace('\n','')
        assert len(seqs) == len(labels), 'MISMATCH'
        Seqs.append(seqs)
        Labels.append(labels)
        # if line is empty
        # end of file is reached
    #     if count >20:
    #         break
    return Seqs,Labels


In [5]:
import pandas as pd
import numpy as np
import re
import torch



Seqs,Labels = get_dataset('./yaamformer/data/YAAM.txt')

def tokenize_protein(sequence):
  #print(sequence)
  sequence = re.sub(r"[UZOB]", "X", sequence)
  sequence = ' '.join(sequence.replace('\n',''))
  return sequence

class NERprotein(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])



In [6]:
sum_seq = 0
for i in Seqs:
  sum_seq += len(i)

  sum_rez = 0
for i in Labels:
  sum_rez += sum(i)

print(f'Residues in total dataset with PTMs {sum_rez/sum_seq}')

0.02109174836588914


In [7]:
yaam = pd.DataFrame({'seq':Seqs,
                     'label':Labels})

In [8]:
yaam['len'] = yaam['seq'].map(len)

In [9]:
max_length=1024

yaam = yaam[yaam['len'] < max_length]

In [10]:
print(len(yaam))

4385


In [11]:
test_label = yaam['label'].values[0]
new_label = np.zeros(1024)
new_label[:len(test_label)]=test_label
new_label

array([0., 0., 0., ..., 0., 0., 0.])

In [12]:
def pad_labels(label,max_length=1024):
  new_label = np.zeros(max_length)
  new_label[:len(label)] = label
  return new_label

In [13]:
#pad the labels for batch training
yaam['label'] = yaam['label'].map(pad_labels)

In [14]:
#tokenize the sequences
from sklearn.model_selection import train_test_split
from transformers import BertForMaskedLM, BertTokenizer, pipeline


In [17]:
train, test = train_test_split(yaam, test_size=0.2)

train = train.reset_index()
test = test.reset_index()

In [19]:
train_encodings = [tokenize_protein(x) for x in train['seq']]
test_encodings = [tokenize_protein(x) for x in test['seq']]

tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False, max_length=1024)
train_encodings = tokenizer(train_encodings, padding='max_length', max_length=1024, truncation=True)
test_encodings = tokenizer(test_encodings, padding='max_length', max_length=1024, truncation=True)#padding=True

train_labels = train['label']
test_labels = test['label']

train_dataset = NERprotein(train_encodings, train_labels)
test_dataset = NERprotein(test_encodings, test_labels)

In [21]:
from transformers import BertModel, BertTokenizer, BertForTokenClassification
import re

model = BertForTokenClassification.from_pretrained("Rostlab/prot_bert",)

Some weights of the model checkpoint at Rostlab/prot_bert were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the

In [22]:
from transformers import DataCollatorForTokenClassification
from transformers import TrainingArguments, Trainer

In [29]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer,padding=True,)

training_args = TrainingArguments(
    output_dir='./',          # output directory to where save model checkpoint
    evaluation_strategy="steps",    # evaluate each `logging_steps` steps
    overwrite_output_dir=True,      
    num_train_epochs=10,            # number of training epochs, feel free to tweak
    per_device_train_batch_size=4, # the training batch size, put it as high as your GPU memory fits
    gradient_accumulation_steps=4,  # accumulating the gradients before updating the weights
    per_device_eval_batch_size=4,  # evaluation batch size
    logging_steps=50,             # evaluate, log and save model checkpoints every 1000 step
    save_steps=50,
    gradient_checkpointing = True,
    # load_best_model_at_end=True,  # whether to load the best model (in terms of loss) at the end of training
    # save_total_limit=3,           # whether you don't have much space so you let only 3 model weights saved in the disk
)


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)


using `logging_steps` to initialize `eval_steps` to 50
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [30]:
#trainer.train_dataset[0]

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 3508
  Num Epochs = 10
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 4
  Total optimization steps = 2190
/usr/local/lib/python3.7/dist-packages/transformers/data/data_collator.py:317: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sequence_length = torch.tensor(batch["input_ids"]).shape[1]
/usr/local/lib/python3.7/dist-packages/transformers/data/data_collator.py:328: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = {k: torch.tensor(v, dtype=torch.int64) for k, v in batch.items()}


Step,Training Loss,Validation Loss
50,0.108700,0.094599
100,0.080800,0.079664
150,0.072500,0.069943
200,0.068300,0.064946
250,0.061000,0.064817


***** Running Evaluation *****
  Num examples = 877
  Batch size = 4
Saving model checkpoint to ./checkpoint-50
Configuration saved in ./checkpoint-50/config.json
Model weights saved in ./checkpoint-50/pytorch_model.bin
/usr/local/lib/python3.7/dist-packages/transformers/data/data_collator.py:317: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sequence_length = torch.tensor(batch["input_ids"]).shape[1]
/usr/local/lib/python3.7/dist-packages/transformers/data/data_collator.py:328: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = {k: torch.tensor(v, dtype=torch.int64) for k, v in batch.items()}
***** Running Evaluation *****
  Num examples = 877
  Batch size = 4
Saving model checkpoint